In [212]:
import pandas as pd

df_bb = pd.read_csv('./data/imdb_barbie_Uncleaned.csv', header=None)
df_op = pd.read_csv('./data/imdb_oppenhimmer_Uncleaned.csv', header=None)
df_bb.drop_duplicates(inplace=True)
df_op.drop_duplicates(inplace=True)


# Regular Expression

In [200]:
import re

score_Reg = '(\d{1,2})\/\d{1,2}\s.*\s'
title_Reg = '\d{1,2}\/\d{1,2}\s(.*)'
people_found_helpful_Reg = '([\d,]*) out of ([\d,]*) found this helpful'
username_Reg = '\d{1,2}\/\d{1,2}\s.*\s([A-Za-z0-9_-]+)\d{2}\s[January|February|March|April|June|July|August|September|October|November|December]'
date_Reg = '(\d{1,2}\s\w+\s\d{4})'
review_Reg = '\d{1,2}\s\w+\s\d{4}\s([\s\S]*)\s[\d,]* out of [\d,]* found this helpful'

In [201]:
# Regular Expression test code

test_text = df_bb.iloc[18][0]

print(test_text)

res = re.findall(score_Reg, test_text)
print(res)

res = re.findall(title_Reg, test_text)
print(res)

res = re.findall(username_Reg, test_text)
print(res)

res = re.findall(date_Reg, test_text)
print(res)

res = re.findall(people_found_helpful_Reg, test_text)
print(res)

res = re.findall(review_Reg, test_text)
print(res)

Honestly - not funny and light enough
imdbmovieguy21 July 2023
I really enjoyed the first 20 minutes of the movie. It was very upbeat, positive and light. However, things soon became negative about 20 - 25 minutes into the movie. I couldn't stand it. Barbie's world and attitude was a constant bummer. Ken's attitude was sour. For the next hour (and more) the movie just lived in negativity.

I bought tickets to this show a month ago. And, I saw it the day before the official opening day. I was very excited about this movie. And, I was playing the music (Dua Lipa) song almost every day. I was really excited for this movie - and was looking forward to something very positive. But, the vast majority of the movie was the "conflict" part of the writing. It was just too much! The movie didn't need to go that way, that far, that fast. I would have preferred a more shallow and fun Barbie movie vs. This Barbie movie filled with over an hour of conflict.
358 out of 799 found this helpful. Was this

In [214]:
# Clean every row in data set

# cleand_df_bb = pd.DataFrame(columns=['score', 'title', 'username', 'date', 'people_found_helpful', 'people_didnt_find_helpful', 'review'])

def get_cleaned_df(dirty_df):
	cleaned_df = pd.DataFrame(columns=['score', 'title', 'username', 'date', 'people_found_helpful', 'review'])
    # cleaned_df = pd.DataFrame(columns=['score', 'title', 'username', 'date', 'people_found_helpful', 'people_didnt_find_helpful', 'review'])
	for index in range(len(dirty_df)):
		row = dirty_df.iloc[index][0]
		
		score = re.findall(score_Reg, row)
		title = re.findall(title_Reg, row)
		username = re.findall(username_Reg, row)
		date = re.findall(date_Reg, row)
		people_found_helpful = re.findall(people_found_helpful_Reg, row)
		review = re.findall(review_Reg, row)
		
		if not score:
			score = ['NULL']
		if not title:
			title = ['NULL']
		if not username:
			username = ['NULL']
		if not date:
			date = ['NULL']
		if not people_found_helpful:
			people_found_helpful = ['NULL', 'NULL']
		if not review:
			review = ['NULL']
			
		cleaned_df.loc[index] = [score[0], title[0], username[0], date[0], people_found_helpful[0], review[0]]
	return cleaned_df

In [219]:
cleand_df_bb = get_cleaned_df(df_bb)
cleand_df_bb.drop_duplicates(subset=['username'], inplace=True)

In [220]:
cleand_df_bb

,score,title,username,date,people_found_helpful,review
0,6,Creative but preachy and confusing,Imshortok06,23 July 2023,"(1,007, 1,363)",I'm just going to get straight to the point: I...
1,6,"Beautiful film, but so preachy",LoveofLegacy,21 July 2023,"(1,190, 1,646)","Margot does the best with what she's given, bu..."
2,6,Clever and fun beginning turns into another le...,Binkconn,21 July 2023,"(804, 1,173)","Barbie begins in an irresistible way, as a fun..."
3,8,It was depressing,coxaneesa,24 July 2023,"(152, 242)",Warning: Spoilers
4,9,A Technicolor Dream,heatherhilgers,24 July 2023,"(20, 26)","Wow, this movie was a love letter to cinema. F..."
...,...,...,...,...,...,...
792,2,Barbie-not good,thongman,22 July 2023,"(5, 17)",The only reason this movie even gets a 2 from ...
793,5,Feminism but the rest?,iyaseminkose,23 July 2023,"(1, 6)","I'm NOT a Barbie girl in NOT the Barbie World,..."
794,3,"Great sets, great execution but horrible story...",DaHermitKid,28 July 2023,"(1, 5)","Firstly, the technical execution is top notch...."
795,2,Just meh.,apriljosey-29034,27 July 2023,"(0, 1)",Warning: Spoilers


In [221]:
cleand_df_op = get_cleaned_df(df_op)
cleand_df_op.drop_duplicates(subset=['username'], inplace=True)

In [222]:
cleand_df_op

,score,title,username,date,people_found_helpful,review
0,9,Murphy is exceptional,Orlando_Gardner,19 July 2023,"(1,413, 1,597)",You'll have to have your wits about you and yo...
1,8,"A challenging watch to be sure, but a worthwhi...",Jeremy_Urquhart,20 July 2023,"(685, 819)",One of the most anticipated films of the year ...
2,10,A brilliantly layered examination of a man thr...,MrDHWong,20 July 2023,"(556, 691)","""Oppenheimer"" is a biographical thriller film ..."
3,10,Nolan delivers a powerfull biopic that shows t...,and_mikkelsen,20 July 2023,"(358, 457)",This movie is just... wow! I don't think I hav...
4,10,A Masterpiece,mohameddawoud-26019,19 July 2023,"(670, 833)",I may consider myself lucky to be alive to wat...
...,...,...,...,...,...,...
2031,7,"Huge story, but production fails sometimes",andreverschoor,27 July 2023,"(1, 3)","Loved the movie, loved the actors efforts. Aft..."
2032,6,Tweaked,fmacr,25 July 2023,"(1, 7)","I suppose these reviews are tweaked, otherwise..."
2033,6,Just a 6 for a near perfect film.,m-nicolaou-86-661254,24 July 2023,"(0, 1)",Ok let me start by the obvious ratings which w...
2034,6,No need to be 3 hours,scottahaynie,24 July 2023,"(0, 0)",Acting was tremendous and the true story behin...
